# Práctica 3: MongoDB - Limpieza de datos
- Martínez Ostoa Néstor Iván
- Bases de Datos No Estructuradas, 0600
- LCD, IIMAS, UNAM

In [11]:
import pandas as pd
import numpy as np
import json
from mongo_client import MongoDBClient

## Lectura de datos

In [2]:
base_path = 'citybike-data/'
df = pd.read_csv(base_path + '1.csv')
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753231,-73.970325,504,1 Ave & E 15 St,40.732219,-73.981656,16950,Customer,\N,0
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.749718,-74.002950,459,W 20 St & 11 Ave,40.746745,-74.007756,19816,Customer,\N,0
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,1980,2
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.718939,-73.992663,499,Broadway & W 60 St,40.769155,-73.981918,16063,Customer,\N,0
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,1986,1


## Verificación de estaciones

Verificamos que las estaciones de origen y destino sean las mismas

In [3]:
for i in range(1,9):
    df = pd.read_csv(base_path + str(i) + '.csv')
    start_ids = sorted(df['start station id'].unique())
    end_ids = sorted(df['end station id'].unique())
    print(start_ids == end_ids)

True
True
True
True
True
True
True
True


## Escritura de estaciones de bicicletas

In [4]:
bike_ids = df['start station id'].unique()
ids = []
names = []
lats = []
lons = []
for id in bike_ids:
    row = df[df['start station id'] == id].iloc[0, [3,4,5,6]]
    ids.append(row[0])
    names.append(row[1])
    lats.append(row[2])
    lons.append(row[3])
bike_stations = pd.DataFrame(
    {'id': ids, 'bs_name': names, 'latitude':lats, 'longitude':lons}
)

In [5]:
bike_stations

,id,bs_name,latitude,longitude
0,294,Washington Square E,40.730494,-73.995721
1,285,Broadway & E 14 St,40.734546,-73.990741
2,247,Perry St & Bleecker St,40.735354,-74.004831
3,357,E 11 St & Broadway,40.732618,-73.991580
4,401,Allen St & Rivington St,40.720196,-73.989978
...,...,...,...,...
324,2001,7 Ave & Farragut St,40.698921,-73.973330
325,533,Broadway & W 39 St,40.752996,-73.987216
326,362,Broadway & W 37 St,40.751726,-73.987535
327,267,Broadway & W 36 St,40.750977,-73.987654


In [6]:
f = open(base_path + "bike_stations.json", "w")
for _, row in bike_stations.iterrows():
    bd = {
        'loc': {
            'coordinates': [row.longitude, row.latitude],
            'type':'Point'
        },
        'name': row.bs_name,
        'id': row.id
    }
    f.write(json.dumps(bd) + '\n')
f.close()

## Escritura y limpieza de datos de viajes

In [4]:
def transform_df(df):
    df = df.rename(columns={
        'tripduration':'trip_duration',
        'starttime':'start_time',
        'stoptime':'stop_time',
        'start station id': 'start_station_id',
        'end station id': 'end_station_id'
    })
    return df.loc[:, ['trip_duration', 'start_time', 'stop_time', 'start_station_id', 'end_station_id']]

In [3]:
# Escritura
base_path = 'citybike-data/'
for i in range(1, 9):
    df = pd.read_csv(base_path + f"{i}.csv")
    df = transform_df(df)    
    df.to_json(base_path + f"{i}.json", orient='records', lines=True)

## Simulación de viajes

In [12]:
base_path = 'citybike-data/'
df = transform_df(pd.read_csv(base_path + '1.csv'))
for i in range(2, 9):
    temp = pd.read_csv(base_path + f"{i}.csv")
    temp = transform_df(temp)
    df = pd.concat([df, temp])
print(str(df.shape[0]) + ' registros')

5562321 registros


In [9]:
client = MongoDBClient(db_name='citi-bike')

In [14]:
# Empezando desde la biblioteca Stephen A. Schwarzman de NY
time = 30
latitude = 40.75350713285891
longitude = -73.98233520342376
#round_trip = False
#time_of_query =

In [ ]:
class BikeStation:
    def __init__(self, bike_station_id, name, latitude, longitude):
        self.bike_station_id = bike_station_id
        self.name = name
        self.latitude = latitude
        self.longitude = longitude

In [30]:
def find_nearest_station(lat, lon):
    stations = client.db['stations']
    result = stations.aggregate([
        {
            "$geoNear": {
                "near": {"type":"Point", "coordinates": [lon, lat]},
                "key": "loc",
                "distanceField":"dist",
                "spherical":True
            }
        },
        {"$limit":10}
    ])
    return result

In [32]:
for s in find_nearest_station(40.752062306999996,-73.9816324043):
    print(s)

{'_id': ObjectId('60b3e3a8d16447115b91e01e'), 'loc': {'coordinates': [-73.9816324043, 40.752062306999996], 'type': 'Point'}, 'name': 'E 40 St & 5 Ave', 'id': 153, 'dist': 0.0}
{'_id': ObjectId('60b3e3a8d16447115b91df4c'), 'loc': {'coordinates': [-73.98338988, 40.75038009], 'type': 'Point'}, 'name': 'W 37 St & 5 Ave', 'id': 485, 'dist': 238.81512690970024}
{'_id': ObjectId('60b3e3a8d16447115b91df8c'), 'loc': {'coordinates': [-73.97798848, 40.75149263], 'type': 'Point'}, 'name': 'Pershing Square S', 'id': 517, 'dist': 313.76390823537673}
{'_id': ObjectId('60b3e3a8d16447115b91df52'), 'loc': {'coordinates': [-73.97770164, 40.75188406], 'type': 'Point'}, 'name': 'Pershing Square N', 'id': 519, 'dist': 332.06981536259127}
{'_id': ObjectId('60b3e3a8d16447115b91dfea'), 'loc': {'coordinates': [-73.9779874, 40.75320159], 'type': 'Point'}, 'name': 'E 43 St & Vanderbilt Ave', 'id': 318, 'dist': 332.5116894421663}
{'_id': ObjectId('60b3e3a8d16447115b91dff8'), 'loc': {'coordinates': [-73.98014437, 4